In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import dill
from pydrake.geometry import StartMeshcat

from iiwa_batter import PACKAGE_ROOT, CONTROL_DT, PITCH_DT, CONTACT_DT, NUM_JOINTS
from iiwa_batter.robot_constraints.get_joint_constraints import JOINT_CONSTRAINTS
from iiwa_batter.physics import (
    PITCH_START_POSITION,
    FLIGHT_TIME_MULTIPLE,
    find_ball_initial_velocity,
)
from iiwa_batter.trajectory_library import LIBRARY_POSITIONS, LIBRARY_SPEEDS_MPH

from iiwa_batter.swing_simulator import setup_simulator, run_swing_simulation, reset_systems
from iiwa_batter.swing_optimization.stochastic_gradient_descent import make_torque_trajectory
from iiwa_batter.swing_optimization.full_trajectory import full_trajectory_reward
from iiwa_batter.swing_optimization.swing_impact import calculate_plate_time_and_ball_state, dummy_torque_trajectory

Loaded joint constraints for KUKA robots


In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [ ]:
robot_constraints = JOINT_CONSTRAINTS["iiwa14"]

simulator, diagram = setup_simulator(torque_trajectory={0:np.ones(NUM_JOINTS)*40}, model_urdf="iiwa14", dt=PITCH_DT, add_contact=False, robot_constraints=robot_constraints, meshcat=meshcat)

status_dict = run_swing_simulation(
    simulator=simulator,
    diagram=diagram,
    start_time=0,
    end_time=4.0,
    initial_joint_positions=np.array([0] * NUM_JOINTS),
    initial_joint_velocities=np.array([0] * NUM_JOINTS),
    initial_ball_position=PITCH_START_POSITION,
    initial_ball_velocity=np.zeros(3),
    meshcat=meshcat,
    record_state=True,
)

In [ ]:
torque_trajectory={
    0: np.ones(NUM_JOINTS)*-40,
    1: np.ones(NUM_JOINTS)*40
}

simulator, diagram = setup_simulator(torque_trajectory, model_urdf="iiwa14", dt=PITCH_DT, add_contact=True, meshcat=meshcat)

from iiwa_batter.swing_optimization.stochastic_gradient_descent import find_initial_positions

initial_joint_positions = find_initial_positions(simulator, diagram, robot_constraints, 1)

reset_systems(diagram)

run_swing_simulation(
    simulator=simulator,
    diagram=diagram,
    start_time=0,
    end_time=2*CONTACT_DT,
    initial_joint_positions=np.array([0] * NUM_JOINTS),
    initial_joint_velocities=np.array([0] * NUM_JOINTS),
    initial_ball_position=PITCH_START_POSITION,
    initial_ball_velocity=np.zeros(3),
    meshcat=meshcat,
)


In [ ]:
robot = "iiwa14"
optimization_name = "fine"
LIBRARY_INDEX = 3
save_directory = f"{PACKAGE_ROOT}/../trajectories/{robot}/{90}_{LIBRARY_POSITIONS[LIBRARY_INDEX]}"
trajectory_source = f"{save_directory}/{optimization_name}.dill"

with open(trajectory_source, "rb") as f:
    results_dict = dill.load(f)

initial_joint_positions = results_dict["best_initial_position"]
control_vector = results_dict["best_control_vector"]
dt = results_dict["optimized_dt"]
expected_reward = results_dict["final_best_reward"]
print(f"Expected reward: {expected_reward}")

robot_constraints = JOINT_CONSTRAINTS[robot]

ball_initial_velocity, ball_time_of_flight = find_ball_initial_velocity(90, LIBRARY_POSITIONS[LIBRARY_INDEX])
torque_trajectory = make_torque_trajectory(control_vector, ball_time_of_flight)
simulator, diagram = setup_simulator(torque_trajectory=torque_trajectory, model_urdf=robot, dt=dt, meshcat=meshcat, robot_constraints=robot_constraints)

status_dict = run_swing_simulation(
    simulator=simulator,
    diagram=diagram,
    start_time=0,
    end_time=ball_time_of_flight+0.2,
    initial_joint_positions=initial_joint_positions,
    initial_joint_velocities=np.zeros(NUM_JOINTS),
    initial_ball_position=PITCH_START_POSITION,
    initial_ball_velocity=ball_initial_velocity,
    meshcat=meshcat,
)

print(status_dict)

# reward = full_trajectory_reward(
#     simulator,
#     diagram,
#     initial_joint_positions,
#     control_vector,
#     ball_initial_velocity,
#     ball_time_of_flight,
# )

# print(reward)



In [ ]:
np.random.seed(0)

robot = "iiwa14"
target_velocity_mph = LIBRARY_SPEEDS_MPH[0]
target_position = LIBRARY_POSITIONS[0]
#optimization_name = "test_save_load_consistency_single_swing_impact"
#save_directory = f"{PACKAGE_ROOT}/tests/trajectories"
optimization_name = "impact_1"
save_directory = f"{PACKAGE_ROOT}/../trajectories/{robot}/{target_velocity_mph}_{target_position}"
test_dt = CONTACT_DT

plate_time, plate_ball_postion, plate_ball_velocity = calculate_plate_time_and_ball_state(target_velocity_mph, target_position, test_dt)

with open(f"{save_directory}/{optimization_name}.dill", "rb") as f:
    results_dict = dill.load(f)

initial_joint_positions = results_dict["best_joint_positions"]
initial_joint_velocities = results_dict["best_joint_velocities"]
simulator, diagram = setup_simulator(
    torque_trajectory = dummy_torque_trajectory(plate_time),
    model_urdf=robot,
    dt=test_dt,
    robot_constraints=JOINT_CONSTRAINTS[robot],
    meshcat=meshcat
)
status_dict = run_swing_simulation(
    simulator=simulator,
    diagram=diagram,
    start_time=plate_time,
    end_time=max(dummy_torque_trajectory(plate_time).keys()),
    initial_joint_positions=initial_joint_positions,
    initial_joint_velocities=initial_joint_velocities,
    initial_ball_position=plate_ball_postion,
    initial_ball_velocity=plate_ball_velocity,
    meshcat=meshcat,
)